In [1]:
import pandas as pd
import datetime
from Utils import vkapi as vk
import numpy as np
from multiprocessing.dummy import Pool as ThreadPool
import time
import logging
LOG_FILENAME = 'PostGrab.log'
# Add the log message handler to the logger
logger = logging.getLogger('Grabber')
logger.setLevel(logging.DEBUG)

formatter=logging.Formatter('%(levelname)-8s [%(asctime)s] (%(module)s : %(funcName)s) %(message)s')
handler = logging.handlers.RotatingFileHandler(LOG_FILENAME, maxBytes=100*2**10, backupCount=5)
handler.setFormatter(formatter)

logger.addHandler(handler)
# logging.basicConfig(filename='PostGrab.log',format='%(levelname)-8s [%(asctime)s] (%(module)s : %(funcName)s) %(message)s',level=logging.INFO)
logger.info("application started")

In [2]:
def __worker__(owner_id, api, pool, count=100):
    posts_list = api.wall.get(**{'owner_id': owner_id, 'count': 100})
    if posts_list is None:
        return pd.DataFrame()
    count = posts_list[0]
    posts_list = pd.DataFrame(posts_list[1:])
    if count <= 100:
        return posts_list
    return posts_list.append(pool.map(lambda param: pd.DataFrame(api.wall.get(**param)[1:]), [
        {'owner_id': owner_id, 'count': 100 if count >= 100 else count, 'offset': offset}
        for offset in range(100, count, 100)
        ]), ignore_index=True)
def get_posts(owner_ids: list, pool, count=None, token=None):
    """Сбор постов"""
    api = vk.API((vk.Session(access_token=token) if token else vk.Session()))
    return pd.DataFrame().append(
        list(
            pool.map(
                lambda owner_id: __worker__(owner_id, api, pool, int(count) if count else count),
                owner_ids
            )
        )
        ,ignore_index=True
    )

In [ ]:
while True:
    with ThreadPool(450) as pool:
        l=get_posts(np.random.randint(1, 132000000, 100), pool)
        texts=l[l['text']!=0]['text']
        today=time.localtime()
        logger.info("len:{}".format(len(texts)))
        
        with open('./csv/posts_{}-{}-{}.csv'.format(today[2],today[1],today[0]), 'a') as f:
            texts.to_csv(f, index=False)